In [7]:
# load data from catalog
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import math
import time
import matplotlib.pyplot as plt
from typing import Dict
from torch.utils.data import TensorDataset, DataLoader
from captum.attr import IntegratedGradients

# Check if CUDA is available, if not, check for MPS (Metal Performance Shaders for Apple Silicon), otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [2]:
# load aligned_df from path
file_path = "aligned_df.pq"
data_set = pd.read_parquet(file_path)


In [3]:

# Hyper-parameters 

num_classes = 1
num_epochs = 5
batch_size = 32
learning_rate = 0.0001


# input_size = 5
sequence_length = 100 # the window it trains with can be selected
hidden_size = 32
# hidden_size = 256 
num_layers = 2
step_size = 1

In [11]:
# create sequences for each experiment
# sequence_length = 50  # Example sequence length

# features = ['timestamp', 'A1_Sensor', 'A1_Sensor_diff', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Sensor_norm', 'A1_Resistance_norm']
features = ['timestamp_bin', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Resistance_norm']
target_column = 'resistance_ratio'

input_size = len(features)  # Number of features

# Initialize lists to hold sequences and targets
sequences = []
targets = []

# ---------padding----------
padding_length = 50  # Number of timesteps to pad at the end

# Group by 'exp_no' and create sequences for each group
for _, group in data_set.groupby('exp_no'):
    # Only select the rows with the relevant columns
    data = group[features].values
    target_data = group[target_column].values
    
    # Pad the end of the dataset with zeros for features
    pad_feature = np.zeros((padding_length, len(features)))
    data = np.vstack((data, pad_feature))
    
    # Optionally, pad the end of the dataset with zeros or a specific value for targets
    pad_target = np.zeros(padding_length)
    target_data = np.concatenate((target_data, pad_target))

    # Create sequences
    for i in range(len(data) - sequence_length):
        # Extract the sequence of features and the corresponding target
        sequence = data[i:(i + sequence_length)]
        target = target_data[i + sequence_length - 1]  # Target aligned with the end of the sequence
        
        sequences.append(sequence)
        targets.append(target)
# ---------padding----------

# ---------no padding----------


# # Group by 'exp_no' and create sequences for each group
# for _, group in data_set.groupby('exp_no'):
#     # Only select the rows with the relevant columns
#     data = group[features].values
#     target_data = group[target_column].values
    
#     # Create sequences
#     for i in range(len(group) - sequence_length):
#         # Extract the sequence of features and the corresponding target
#         sequence = data[i:(i + sequence_length)]
#         target = target_data[i + sequence_length]  # Target is the next record
        
#         sequences.append(sequence)
#         targets.append(target)


# ---------no padding----------

# Convert lists to numpy arrays
sequences_np = np.array(sequences)
targets_np = np.array(targets)

sequences_train, sequences_test, targets_train, targets_test = train_test_split(
    sequences_np, targets_np, test_size=0.2, random_state=42)


# Initialize the scaler
scaler = StandardScaler()

# Reshape, fit, and transform the training data
n_samples_train, sequence_length, n_features = sequences_train.shape
sequences_train_reshaped = sequences_train.reshape(-1, n_features)
scaler.fit(sequences_train_reshaped)  # Fit only on training data
sequences_train_scaled = scaler.transform(sequences_train_reshaped).reshape(n_samples_train, sequence_length, n_features)

# Transform the testing data
n_samples_test, _, _ = sequences_test.shape
sequences_test_reshaped = sequences_test.reshape(-1, n_features)
sequences_test_scaled = scaler.transform(sequences_test_reshaped).reshape(n_samples_test, sequence_length, n_features)

# Convert numpy arrays to float32 before converting to PyTorch tensors
sequences_np = sequences_np.astype(np.float32)
targets_np = targets_np.astype(np.float32)

train_sequences_tensor = torch.tensor(sequences_train_scaled, dtype=torch.float32)
test_sequences_tensor = torch.tensor(sequences_test_scaled, dtype=torch.float32)

train_targets_tensor = torch.tensor(targets_train, dtype=torch.float32)
test_targets_tensor = torch.tensor(targets_test, dtype=torch.float32)

# Create TensorDatasets
train_dataset = TensorDataset(train_sequences_tensor, train_targets_tensor)
test_dataset = TensorDataset(test_sequences_tensor, test_targets_tensor)

# Create DataLoaders
# batch_size = batch_size  # You can adjust this based on your memory constraints
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [5]:
# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)
        
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
        # x: (n, 28, 28), h0: (2, n, 128)
        
        # Forward propagate RNN
        # out, _ = self.rnn(x, h0)  
        # or:
        out, _ = self.lstm(x, (h0,c0))  
        
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out: (n, 28, 128)
        
        # Decode the hidden state of the last time step
        out = out[:, -1, :]
        # out: (n, 128)
         
        out = self.fc(out)
        # out: (n, 10)
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


In [8]:
# Create a directory for checkpoints if it doesn't exist
checkpoint_dir = './checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Assuming model, train_loader, and test_loader are defined as per your code
# Assuming device selection logic is applied before this snippet
model.to(device)  # Move your model to the GPU if available

criterion = nn.MSELoss()  # For regression tasks
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_losses = []
validation_rmses = []

for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    for sequences_batch, targets_batch in train_loader:
        sequences_batch = sequences_batch.to(device)
        targets_batch = targets_batch.to(device).unsqueeze(-1)
        
        outputs = model(sequences_batch)
        loss = criterion(outputs, targets_batch)
        epoch_losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_training_loss = sum(epoch_losses) / len(epoch_losses)
    training_losses.append(avg_training_loss)
    
    # Validation phase
    model.eval()
    total_val_loss = 0
    count = 0
    with torch.no_grad():
        for sequences_batch, targets_batch in test_loader:
            sequences_batch = sequences_batch.to(device)
            targets_batch = targets_batch.to(device).unsqueeze(-1)
            
            outputs = model(sequences_batch)
            loss = criterion(outputs, targets_batch)
            
            total_val_loss += loss.item()
            count += 1
    
    avg_val_loss = total_val_loss / count
    val_rmse = math.sqrt(avg_val_loss)
    validation_rmses.append(val_rmse)
    
    print(f'Epoch {epoch+1}, Training Loss: {avg_training_loss}, Validation RMSE: {val_rmse}')
    
    # Save checkpoint
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}_{timestamp}.pt')
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'training_loss': avg_training_loss,
        'validation_rmse': val_rmse,
    }, checkpoint_path)
    
    # Optionally, save plots for loss and validation RMSE
    if (epoch + 1) % 5 == 0:  # For example, save plots every 5 epochs
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.plot(range(1, epoch + 2), training_losses, label='Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss Over Time')
        
        plt.subplot(1, 2, 2)
        plt.plot(range(1, epoch + 2), validation_rmses, label='Validation RMSE')
        plt.xlabel('Epoch')
        plt.ylabel('RMSE')
        plt.title('Validation RMSE Over Time')
        
        plt.tight_layout()
        plt.savefig(os.path.join(checkpoint_dir, f'plots_epoch_{epoch+1}_{timestamp}.png'))
        plt.close()


Epoch 1, Training Loss: 0.0179478300479531, Validation RMSE: 0.018447727121241236
Epoch 2, Training Loss: 0.0003660850780614467, Validation RMSE: 0.019212006961797222
Epoch 3, Training Loss: 0.00034945835523202205, Validation RMSE: 0.019126504622256242
Epoch 4, Training Loss: 0.00033930437102849104, Validation RMSE: 0.017670363111789853
Epoch 5, Training Loss: 0.00033468627717593205, Validation RMSE: 0.017843368139127916


In [9]:
# Define the directory where you want to save your metrics
base_directory = "importance_results"
run_directory = os.path.join(base_directory, timestamp)

# Create the directory if it does not exist
if not os.path.exists(run_directory):
    os.makedirs(run_directory)

ig = IntegratedGradients(model)

input_tensor = test_sequences_tensor[10].unsqueeze(0).to(device).float()  # Add batch dimension, move to device, and ensure float32
attributions, delta = ig.attribute(input_tensor, return_convergence_delta=True)
attributions = attributions.float()

# Define the compute_importances function
def compute_importances(model, input_sequence):
    # Initialize IntegratedGradients with the model
    ig = IntegratedGradients(model)
    
    # Ensure the input sequence tensor is in float32 and add a batch dimension
    input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Compute attributions using Integrated Gradients
    attributions, delta = ig.attribute(input_tensor, return_convergence_delta=True)
    
    # Ensure the returned attributions are in float32
    attributions = attributions.float()
    
    # Return the attributions and delta as numpy arrays
    return attributions.detach().cpu().numpy(), delta.detach().cpu().numpy()
# Assuming 'model', 'device', and 'compute_importances' function are defined as before

# Select only the group for exp_no = 0
group = data_set[data_set['exp_no'] == 0]

total_timesteps = len(group)  # Total timesteps for exp_no = 0
number_of_features = len(features)  # Assuming 'features' is a list of feature names
sequence_length = 100  # Assuming sequence length is defined

# Initialize arrays for cumulative importances and contributions
cumulative_importances = np.zeros((total_timesteps, number_of_features))
contributions = np.zeros(total_timesteps)

all_importances = []  # List to store all importances for sequences
for i in range(total_timesteps - sequence_length + 1):
    sequence = group[features].values[i:(i + sequence_length)]
    # Compute importances for the sequence
    importances, _ = compute_importances(model, sequence)
    all_importances.append(importances)

# Aggregate the importances in a temporary array first
for idx, imp in enumerate(all_importances):
    start_idx = idx
    end_idx = start_idx + sequence_length

    imp_reshaped = imp.squeeze(0)

    # Directly update the cumulative importances and contributions
    cumulative_importances[start_idx:end_idx, :] += imp_reshaped
    contributions[start_idx:end_idx] += 1

# Calculate average importances by avoiding division by zero
avg_importances = np.zeros_like(cumulative_importances)
non_zero_contributions = contributions > 0
avg_importances[non_zero_contributions, :] = cumulative_importances[non_zero_contributions, :] / contributions[non_zero_contributions, None]

# Plot the average importances for exp_no = 0
timesteps = np.arange(total_timesteps)  # An array of timesteps from 0 to the last one

for feature_idx in range(number_of_features):
    plt.plot(timesteps, avg_importances[:, feature_idx], label=f'Feature {feature_idx + 1}')

plt.legend()
plt.xlabel('Timestep')
plt.ylabel('Average Feature Importance')
plt.title('Average Feature Importances Over Time for Exp 0')

# Save the plot in the run-specific directory with a meaningful name
plt.savefig(os.path.join(run_directory, 'average_feature_importances.png'))
plt.close()  # Close the plot to avoid displaying it inline if not needed

# If you need to save the numeric data as well:
np.save(os.path.join(run_directory, 'avg_importances.npy'), avg_importances)


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

---

In [ ]:


# Define the compute_importances function
def compute_importances(model, input_sequence):
    # Initialize IntegratedGradients with the model
    ig = IntegratedGradients(model)
    
    # Ensure the input sequence tensor is in float32 and add a batch dimension
    input_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Compute attributions using Integrated Gradients
    attributions, delta = ig.attribute(input_tensor, return_convergence_delta=True)
    
    # Ensure the returned attributions are in float32
    attributions = attributions.float()
    
    # Return the attributions and delta as numpy arrays
    return attributions.detach().numpy(), delta.detach().numpy()

# Dictionary to store aggregated importances for each experiment
experiment_importances = {}

for exp_no, group in data_set.groupby('exp_no'):
    # Initialize a list to store importances for all sequences in this experiment
    all_importances = []
    
    # Compute importances for each sequence within this experiment
    for i in range(len(group) - sequence_length):
        sequence = group[features].values[i:(i + sequence_length)]
        # Assuming sequence is in the correct shape for the model
        importances = compute_importances(model, sequence)
        all_importances.append(importances)
    
    # Aggregate the importances across all sequences for this experiment
    # Here, we're taking the mean, but you could also sum them or use another method
    aggregated_importances = np.mean(all_importances, axis=0)
    
    # Store the aggregated importances in the dictionary
    experiment_importances[exp_no] = aggregated_importances

